In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import save_npz

In [ ]:
data =  pd.read_csv("/home/tuanh/M1/projet/code/IMDb_All_Genres_etf_clean1.csv")
print(data.columns)

Index(['Movie_Title', 'Year', 'Director', 'Actors', 'Rating', 'Runtime(Mins)',
       'Censor', 'Total_Gross', 'main_genre', 'side_genre'],
      dtype='object')


In [ ]:
print(data.head().loc[2])

Movie_Title          The Lord of the Rings: The Return of the King
Year                                                          2003
Director                                             Peter Jackson
Actors           Elijah Wood, Viggo Mortensen, Ian McKellen, Or...
Rating                                                         9.0
Runtime(Mins)                                                  201
Censor                                                           U
Total_Gross                                               $377.85M
main_genre                                                  Action
side_genre                                       Adventure,  Drama
Name: 2, dtype: object


In [ ]:
print(np.count_nonzero(data.isna()))

0


In [ ]:

data["bag_of_words"] = (
    data["Movie_Title"] + " " 
    +data["Director"] + " " 
    + data["Actors"] + " " 
    + data["main_genre"] + " " 
    + data["side_genre"]
)

data[["Movie_Title", "bag_of_words"]].head()


,Movie_Title,bag_of_words
0,Kantara,"Kantara Rishab Shetty Rishab Shetty, Sapthami ..."
1,The Dark Knight,The Dark Knight Christopher Nolan Christian Ba...
2,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King ...
3,Inception,"Inception Christopher Nolan Leonardo DiCaprio,..."
4,The Lord of the Rings: The Two Towers,The Lord of the Rings: The Two Towers Peter Ja...


In [ ]:
train_df = data
test_df = data.iloc[:5]

print(len(train_df), len(test_df))


5562 5


In [ ]:
vectorizer = TfidfVectorizer( 
)


tfidf_matrix= vectorizer.fit_transform(train_df["bag_of_words"])





In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_for_test_index(idx_in_test, top_k=5, alpha=0.2):
    idx_in_test = [1,2,3,4,5]
    query_vec = tfidf_matrix[idx_in_test]
    sims = cosine_similarity(query_vec, tfidf_matrix) 
    ratings = data["Rating"].astype(float).to_numpy()
    r_min = ratings.min()
    r_max = ratings.max()
    rating_norm = (ratings - r_min) / (r_max - r_min)   

    score = (1 - alpha) * sims + alpha * rating_norm  

    sorted_idx = np.argsort(score)[::-1]              

    filtered_idx = []
    for i in sorted_idx:
        if data.index[i] in idx_in_test:
            continue
        filtered_idx.append(i)
        if len(filtered_idx) == top_k:
            break

    top_idx = np.array(filtered_idx)

    recs = data.iloc[top_idx][[
        "Movie_Title", "Year", "main_genre", "side_genre", "Rating", "plot"
    ]].copy()

    recs["similarity"] = sims[top_idx]
    recs["score"] = score[top_idx]

    print("Film de requête (dans le test) :")
    print(data.iloc[idx_in_test])
    print("\nRecommandations (dans le train) :\n")

    return recs


In [ ]:
recs = recommend_for_test_index([4,6,7], top_k=5)
print(recs)
#

Film de requête (dans le test) :
                                     Movie_Title  Year main_genre  \
0                                        Kantara  2022     Action   
1                                The Dark Knight  2008     Action   
2  The Lord of the Rings: The Return of the King  2003     Action   
3                                      Inception  2010     Action   
4          The Lord of the Rings: The Two Towers  2002     Action   

            side_genre  Rating  
0    Adventure,  Drama     9.3  
1        Crime,  Drama     9.0  
2    Adventure,  Drama     9.0  
3   Adventure,  Sci-Fi     8.8  
4    Adventure,  Drama     8.8  

Recommandations (dans le train) :

                      Movie_Title  Year main_genre       side_genre  Rating  \
3450                  777 Charlie  2022  Adventure   Comedy,  Drama     8.9   
65                     Hera Pheri  2000     Action   Comedy,  Crime     8.1   
3453              Soorarai Pottru  2020      Drama            Drama     8.7   
23

In [ ]:
# import pickle
# with open("vectorizer.pkl", "wb") as f:
#     pickle.dump(vectorizer, f)

# from scipy.sparse import save_npz

# save_npz("tfidf_train.npz", tfidf_train)
